In [1]:
import solution_file_processing as sfp
import os
import numpy as np
import pandas as pd
from dask import dataframe as dd

from solution_file_processing.utils.utils import catch_errors
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE, IEA_PALETTE_16, IEA_PALETTE_PLUS, EXTENDED_PALETTE
from solution_file_processing.constants import VRE_TECHS, PRETTY_MODEL_NAMES
from solution_file_processing.timeseries import create_output_11 as create_ts_output_11
from solution_file_processing.timeseries import create_output_4 as create_timeseries_output_4
from solution_file_processing import log
from solution_file_processing.plots import _get_plot_1_variables

# Initialize confi||g with toml file
c = sfp.SolutionFilesConfig('config_files/ukraine/UKR.toml')

# c = sfp.SolutionFilesConfig('config_files/thailand/THA.toml')
ix = pd.IndexSlice  

from solution_file_processing.plots import create_plot_1a, create_plot_2b_ref_plots, create_plot_2_summary, create_plot_10_ts_by_model
from solution_file_processing.plots import create_plot_6_ldc_and_line_plots, create_plot_7_co2_savings, create_plot_8_services, create_plot_9_av_cap

[11:25:28 00:00] I:119:__init__ - Logging to C:\Users\hart_c\showcase\solution-file-processing\logs\20240911_112528-UKR.log.
[11:25:29 00:00] I:157:__init__ - Initialized SolutionFilesConfig for config_files/ukraine/UKR.toml.


In [6]:
### Check if there are any files to convert in folder and all subfolders
soln_zip_files = [Path(os.path.join(root, name))
        for root, dirs, files in os.walk(self.c.DIR_04_SOLUTION_FILES)
        for name in files
            if name.endswith((".zip", ".ZIP"))]

root_path = Path(self.c.DIR_04_SOLUTION_FILES)
### Move all files to root folder
for f in soln_zip_files:
    if f.parent != root_path:
        # Move the file to the correct directory
        shutil.move(f, root_path / f.name)
        if len(os.listdir(f.parent)) == 0:
            os.rmdir(f.parent)

In [38]:
IEA_PALETTE_PLUS

{'grey5': '#f2f2f2',
 'grey10': '#e6e6e6',
 'pl': '#b187ef',
 'bl': '#49d3ff',
 'tl': '#00e0e0',
 'gl': '#68f394',
 'yl': '#fff45a',
 'ol': '#ffb743',
 'rl': '#ff684d',
 'grey40': '#949494',
 'grey50': '#6f6f6f',
 'p': '#af6ab1',
 'b': '#3e7ad3',
 't': '#00ada1',
 'g': '#1dbe62',
 'y': '#fed324',
 'o': '#f1a800',
 'r': '#e34946',
 'grey20': '#afafaf',
 'black': '#000000',
 'white': '#ffffff',
 'iea_b': '#0044ff',
 'iea_b50': '#80a2ff',
 'brown': '#6E4F00',
 '0': '#393b79',
 '1': '#5254a3',
 '2': '#6b6ecf',
 '3': '#9c9ede',
 '4': '#637939',
 '5': '#8ca252',
 '6': '#b5cf6b',
 '7': '#cedb9c',
 '8': '#8c6d31',
 '9': '#bd9e39',
 '10': '#e7ba52',
 '11': '#e7cb94',
 '12': '#843c39',
 '13': '#ad494a',
 '14': '#d6616b',
 '15': '#e7969c',
 '16': '#7b4173',
 '17': '#a55194',
 '18': '#ce6dbd',
 '19': '#de9ed6',
 '20': '#3182bd',
 '21': '#6baed6',
 '22': '#9ecae1',
 '23': '#c6dbef',
 '24': '#e6550d',
 '25': '#fd8d3c',
 '26': '#fdae6b',
 '27': '#fdd0a2',
 '28': '#31a354',
 '29': '#74c476',
 '30': '#

In [ ]:
model_path = Path('Y:/Modelling/Ukraine/2023_UKR_ST_Security/')
extract_plexos_LT_results(model_path=model_path)